In [ ]:
pip install python_speech_features

Remove Silence



In [ ]:
import os
import numpy as np

def remove_silence(audio):
  audio_m=[]
  for i in range(len(audio)):
    if (abs(audio[i])>10):
      audio_m.append(audio[i])
  audio_m = np.asarray(audio_m)
  return audio_m


Extract_Feature

In [ ]:
import numpy as np
from python_speech_features import mfcc as MFCC
from python_speech_features import delta
from sklearn import preprocessing
from scipy.io import wavfile
import os

def extract_featrue(audio,fs):
    mfcc_feature = MFCC(audio, fs,)
    mfcc = preprocessing.scale(mfcc_feature)
    mfcc_delta = delta(mfcc_feature, 2)
    mfcc_double_delta = delta(mfcc_delta, 2)
    mfcc_final = np.hstack([mfcc, mfcc_delta, mfcc_double_delta])
    return mfcc_final


def get_feature(path,task):
    file_paths = []
    # get all  wav file
    for root, sub, files in os.walk(path):
        files = [f for f in files if f.endswith(".WAV")]
        speaker_file_paths = []
        for file in files:
            speaker_file_paths.append(os.path.join(root, file))
        if speaker_file_paths != []:
            file_paths.append(speaker_file_paths)

    output_path = 'output/op'
    fs=16000
    if task == 'train':
      Feature = np.array([])
      for speaker_file in file_paths:
          for file in speaker_file:
              [fs, audio] = wavfile.read(file)
              audio = remove_silence(audio)
              vector = extract_featrue(audio, fs)
              if Feature.size == 0:
                  Feature = vector
              else:
                  Feature = np.vstack([Feature, vector])
      return Feature
    else:
      Feature = []
      for speaker_file in file_paths:
          for file in speaker_file:
              print(speaker_file)
              print(file)
              [fs, audio] = wavfile.read(file)
              audio = remove_silence(audio)
              vector = extract_featrue(audio, fs)
              Feature.append(vector)
      return Feature

Reading Data and Building UBM

In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
import copy
import math
import seaborn as sns;sns.set()
import matplotlib.pyplot as plt

ubm_guassian_no = 64
feature_dim = 39
train_path = 'drive/My Drive/Colab Notebooks/SIP Project/TIMIT_modified/TRAIN/Train_data'
Feature = get_feature(train_path,'test')
Feature= np.vstack(Feature)
ubm = GaussianMixture(n_components=ubm_guassian_no,covariance_type='diag',max_iter=100)
ubm.fit(Feature)

speaker_path = 'drive/My Drive/Colab Notebooks/SIP Project/TIMIT_modified/TRAIN/target'
spk_feature = get_feature(speaker_path,'test')
n_target_spk = len(spk_feature)
print(f'Number of taret speaker :{n_target_spk}')
test_path = 'drive/My Drive/Colab Notebooks/SIP Project/TIMIT_modified/TRAIN/test'
test_feature = get_feature(test_path,'test')
n_test_spk = len(test_feature)
print(f'Number of test speaker :{n_test_spk}')

Building Speaker specific Model and verifying test speaker

In [ ]:
FP=0
TP=0
FN=0
TN=0

ratio_mat = np.zeros((n_target_spk,n_target_spk))

for k in range(n_target_spk):
  spk_model = GaussianMixture(n_components=ubm_guassian_no,covariance_type='diag')
  spk_model = copy.deepcopy(ubm)
  T = len(spk_feature[k])
  Pr = spk_model.predict_proba(spk_feature[k])
  n = np.zeros(ubm_guassian_no)
  m = np.zeros((ubm_guassian_no,feature_dim))
  v = np.zeros((ubm_guassian_no,feature_dim))

  for i in range(len(spk_feature[k])):
      n = n + Pr[i]
      for j in range(len(Pr[i])):
          m[j] = m[j] + Pr[i][j]*spk_feature[k][i]
          v[j] = v[j] + Pr[i][j]*np.square(spk_feature[k][i])

  for i in range(ubm_guassian_no):
      m[i] = m[i]/n[i]
      v[i] = v[i]/n[i]

  alpha = n/(n+16)

  spk_model.weights_ = ((alpha*n/T) + ((1-alpha)*spk_model.weights_))
  spk_model.weights_ = spk_model.weights_/np.sum(spk_model.weights_)
  for i in range(ubm_guassian_no):
      spk_model.means_[i] = alpha[i]*m[i] + (1-alpha[i])*spk_model.means_[i]
      spk_model.covariances_[i] = alpha[i]*v[i] + (1-alpha[i])*(spk_model.covariances_[i] + np.square(spk_model.means_[i])) - np.square(spk_model.means_[i])

  #Test
  for i in range(len(test_feature)):
    lp_ubm = np.exp(ubm.score_samples(test_feature[i]))
    lp_spk = np.exp(spk_model.score_samples(test_feature[i]))
    lp_ubm = np.sum(lp_ubm)
    lp_spk = np.sum(lp_spk)
    ratio = lp_spk/lp_ubm
    ratio_mat[i//5,k]+=ratio

    if(k==(i//5)):
      if (ratio > 1):
        TP+=1
      else:
        FN+=1
    else:
      if (ratio > 1):
        FP+=1
      else:
        TN+=1


FAR = FP/(n_test_spk*n_target_spk)
FRR = FN/(n_test_spk*n_target_spk)

print(f'TP:{TP}')
print(f'FP:{FP}')
print(f'FN:{FN}')
print(f'TN:{TN}')
print(f'FAR:{FAR}')
print(f'FRR:{FRR}')

#Confusion matrix
cnf_mat = np.zeros((2,2))
cnf_mat[0,0]=TP
cnf_mat[0,1]=FP
cnf_mat[1,0]=FN
cnf_mat[1,1]=TN

ratio_mat = ratio_mat/5
sns.heatmap(ratio_mat,cmap="YlGnBu",annot=False,xticklabels=['1','2','3','4','5','6','7','8','9','10'],yticklabels=['1','2','3','4','5','6','7','8','9','10'])